In [127]:
from plotly.offline import iplot, init_notebook_mode
import plotly.graph_objs as go
import plotly.io as pio
import os
import numpy as np
import pandas as pd
from pathlib import Path
from scipy.interpolate import interp1d
from scipy.stats import linregress


# MYPATH="/Users/weugene/Dropbox/Aslan-Evgenii/media/pictures/Heat-transfer/"
MYPATH="../Heat_transfer_Polymerization/Verif/"

wl=2
MYSTYLE=[dict(color='royalblue', width=wl), dict(color='red', width=wl, dash='dot'),
         dict(color='aqua', width=wl, dash='dash'), dict(color='violet', width=wl, dash='dash'),
         dict(color='lightskyblue', width=wl, dash='dot'), dict(color='orange', width=wl, dash='dot'),
         dict(color='black', width=wl, dash='dash'), dict(color='darkmagenta', width=wl, dash='dash')]
def plot_graph(list_x, list_y, names, xtitle, ytitle, image_name, list_x_fill=[], list_y_fill=[], mode=[], \
			   dash=['solid', 'dot', 'dash', 'longdash'], \
			   colors=['blue', 'red', 'hsv(120,100,100)', 'green', 'black' ], \
			   marker_size=15, xrange =[], yrange = [], \
			   marker_style = ['circle', 'triangle-up', 'triangle-down','square', 'diamond', 'cross',  'x-thin', 'cross-thin' ], \
			   width=1000, height=500, path='./', yanchor='center', y0_anchor=0.01, xanchor='left', x0_anchor=0.3, xlog=False, ylog=False):
	if mode == []:
		for i in range(len(list_x)):
			mode.append('lines+markers')


	while len(marker_style) < len(list_x):
		marker_style[:] = marker_style[:] + marker_style[:]
	figborderlinesize = 0.7
	legborderlinesize = 0.7
	yaxis = dict(
		tickfont = dict(
			family = 'Times New Roman',
			size = 20,
			color = 'black'
		),
		titlefont = dict(
			family = 'Times New Roman',
			size = 25,
			color = 'black'
		),
	)
	xaxis = dict(
		tickfont = dict(
			family = 'Times New Roman',
			size = 20,
			color = 'black'
		),
		titlefont = dict(
			family = 'Times New Roman',
			size = 25,
			color = 'black'
		)
	)

	axis_style = dict(showline=True, gridwidth=1, gridcolor='lightgrey', linewidth=figborderlinesize, linecolor='black', mirror=True, ticks='outside', tickfont = dict(family = 'Times New Roman', size = 20, color = 'black'))
	bg_style = {'plot_bgcolor': 'rgba(255, 255, 255, 1)', 'paper_bgcolor': 'rgba(255, 255, 255, 1)',}


	fig = go.Figure()
	k = len(list_x)
	n_fill = len(list_x_fill)
	if len(list_x_fill) == 2 and len(list_y_fill) == 2:
		fig.add_trace(go.Scatter(x=list_x_fill[1], y=list_y_fill[1], name=names[k+1], mode='lines', fillcolor='blueviolet', line_color='blueviolet', fill='tozeroy')) # fill to trace0 y
		fig.add_trace(go.Scatter(x=list_x_fill[0], y=list_y_fill[0], name=names[k], mode='lines', fillcolor='lightsteelblue',     line_color='indigo', fill='tozeroy')) # fill down to xaxis
	for i,x in enumerate(list_x):
		print('Plot curve number:', i)
		y = np.asarray(list_y[i])
		fig.add_trace(go.Scatter(x=x, y=y, name=names[i],
								 mode=mode[i],
								 marker=dict(
									 size=marker_size,
									 line=dict(width=1)
								 ),
								 marker_symbol=marker_style[i],
								 line=dict(width=2, dash=dash[i]),
								 textfont=dict(
									 family="Times New Roman",
									 size=18,
									 color="LightSeaGreen")
								 ))
		if colors != []:
			fig['data'][i + n_fill]['marker']['line']['color'] = colors[i]
			fig['data'][i + n_fill]['line']['color'] = colors[i]
	fig.update_layout(
		width = width,
		height = height,
		xaxis_title=xtitle,
		yaxis_title=ytitle,
		yaxis = yaxis,
		xaxis = xaxis,
		showlegend=True
	)
	fig.update_layout(bg_style)
	fig.update_xaxes(axis_style)
	fig.update_yaxes(axis_style)
	fig.update_layout(legend=dict(
		bgcolor="White",
		bordercolor="Black",
		borderwidth=figborderlinesize
	))
	fig.update_layout(font=dict(
		family="Times New Roman",
		size=20,
		color="Black"
	))
	fig.update_layout(
		autosize=False,
		margin=dict(
			l=0,
			r=0,
			b=0,
			t=0,
			pad=0.1
		),
		#     paper_bgcolor="LightSteelBlue",
	)
	fig.update_layout(legend=dict(
		yanchor=yanchor,
		y=y0_anchor,
		xanchor=xanchor,
		x=x0_anchor
	))
	if len(xrange) == 2:
		fig.update_xaxes(range=xrange)
	if len(yrange) == 2:
		fig.update_yaxes(range=yrange)
	if xlog:
		fig.update_xaxes(type="log")
	if ylog:
		fig.update_yaxes(type="log")
	fig.show()
	fn = os.path.join(path, image_name)
	print('Write image to file:', fn)
	fig.write_image(str(Path(fn)), engine="kaleido")
	print("Successfully generated:", fn)

        # With Modified Chorin's projection
# graphs('/Users/weugene/basilisk/work/Verification_decaying_vortex/convergence.txt', ['6', '7', '8', '9'], ['-3', '-4', '-5', '-6'], whole=False, prefix="Modified_Chorin_" )
#result no convergence with grid refinement. Pressure field looks bad => bad velocity field

In [137]:
data_w = pd.read_csv("../Heat_transfer_Polymerization/Verif/cylinder_polymerization_wolfram_r=1.2.csv")
data_b = {}
levs = [8,9,10]
for lev in levs:
    data_b[lev] = pd.read_csv("../Heat_transfer_Polymerization/Verif/Jmax=" + str(lev) + "/cylinder_polymerization_basilisk_r=1.2.csv")

tmax = df = data_b[9]['t'].max()
for lev in levs:
    if lev==9:
        df = data_b[8]
        f8T = interp1d(df[df['t']>tmax]['t'], df[df['t']>tmax]['T'])
        f8alpha = interp1d(df[df['t']>tmax]['t'], df[df['t']>tmax]['alpha'])
        
        df = data_b[10]
        f10T = interp1d(df[df['t']>tmax]['t'], df[df['t']>tmax]['T'])
        f10alpha = interp1d(df[df['t']>tmax]['t'], df[df['t']>tmax]['alpha'])
        
        x = np.linspace(tmax+0.01, 23, 1000)

        T9 = 0.5*(f8T(x) + f10T(x))
        alpha9 = 0.5*(f8alpha(x) + f10alpha(x))

        data_b[lev] = pd.concat([data_b[lev], pd.DataFrame(data={'t':x, 'T':T9, 'alpha': alpha9})])
        
        
        

In [133]:
list_x = [data_w['t']]
list_y = [data_w['T']]
names = ['Wolfram']
for lev in levs:
    list_x.append(data_b[lev]['t'])
    list_y.append(data_b[lev]['T'])
    names.append(r'$J_{\max}=' + str(lev) + '$')


plot_graph(list_x, list_y, names, xtitle='t', ytitle='T', image_name='T_t.pdf', list_x_fill=[], list_y_fill=[], mode=['lines', 'lines', 'lines', 'lines'], \
           dash=['solid', 'dot', 'dash', 'longdash'], \
           colors=['blue', 'red', 'hsv(120,100,100)', 'green', 'black' ], \
           marker_size=2, xrange =[0, 21], yrange = [], \
           marker_style = ['circle', 'circle', 'triangle-down','square', 'diamond', 'cross',  'x-thin', 'cross-thin' ], \
           width=1000, height=500, path=MYPATH, yanchor='auto', y0_anchor=0.01, xanchor='left', x0_anchor=0.3)

list_x = [data_w['t']]
list_y = [data_w['alpha']]
for lev in levs:
    list_x.append(data_b[lev]['t'])
    list_y.append(data_b[lev]['alpha'])
    
plot_graph(list_x, list_y, names, xtitle=r'$\Large t$', ytitle=r'$\Large\alpha$', image_name='alpha_t.pdf', list_x_fill=[], list_y_fill=[], mode=['lines', 'lines', 'lines', 'lines'], \
           dash=['solid', 'dot', 'dash', 'longdash'], \
           colors=['blue', 'red', 'hsv(120,100,100)', 'green', 'black' ], \
           marker_size=2, xrange =[0, 21], yrange = [], \
           marker_style = ['circle', 'circle', 'triangle-down','square', 'diamond', 'cross',  'x-thin', 'cross-thin' ], \
           width=1000, height=500, path=MYPATH, yanchor='auto', y0_anchor=0.01, xanchor='left', x0_anchor=0.3)


Plot curve number: 0
Plot curve number: 1
Plot curve number: 2
Plot curve number: 3


Write image to file: ../Heat_transfer_Polymerization/Verif/T_t.pdf
Successfully generated: ../Heat_transfer_Polymerization/Verif/T_t.pdf
Plot curve number: 0
Plot curve number: 1
Plot curve number: 2
Plot curve number: 3


Write image to file: ../Heat_transfer_Polymerization/Verif/alpha_t.pdf
Successfully generated: ../Heat_transfer_Polymerization/Verif/alpha_t.pdf


In [131]:
# data_w = pd.read_csv("../Heat_transfer_Polymerization/Verif/cylinder_polymerization_wolfram_r=1.2.csv")
# data_b = {}
# levs = [8,9,10]
# for lev in levs:
#     data_b[lev] = pd.read_csv("../Heat_transfer_Polymerization/Verif/Jmax=" + str(lev) + "E/cylinder_polymerization_basilisk_r=1.2.csv")
    
# list_x = [data_w['t']]
# list_y = [data_w['T']]
# names = ['Wolfram']
# for lev in levs:
#     list_x.append(data_b[lev]['t'])
#     list_y.append(data_b[lev]['T'])
#     names.append(r'$J_{\max}=' + str(lev) + '$')


# plot_graph(list_x, list_y, names, xtitle='t', ytitle='T', image_name='T_tE.pdf', list_x_fill=[], list_y_fill=[], mode=['lines', 'lines', 'lines', 'lines'], \
#            dash=['solid', 'dot', 'dash', 'longdash'], \
#            colors=['blue', 'red', 'hsv(120,100,100)', 'green', 'black' ], \
#            marker_size=2, xrange =[0,21], yrange = [], \
#            marker_style = ['circle', 'circle', 'triangle-down','square', 'diamond', 'cross',  'x-thin', 'cross-thin' ], \
#            width=1000, height=500, path=MYPATH, yanchor='auto', y0_anchor=0.01, xanchor='left', x0_anchor=0.3)

# list_x = [data_w['t']]
# list_y = [data_w['alpha']]
# for lev in levs:
#     list_x.append(data_b[lev]['t'])
#     list_y.append(data_b[lev]['alpha'])
    
# plot_graph(list_x, list_y, names, xtitle=r'$\Large t$', ytitle=r'$\Large\alpha$', image_name='alpha_tE.pdf', list_x_fill=[], list_y_fill=[], mode=['lines', 'lines', 'lines', 'lines'], \
#            dash=['solid', 'dot', 'dash', 'longdash'], \
#            colors=['blue', 'red', 'hsv(120,100,100)', 'green', 'black' ], \
#            marker_size=2, xrange =[0,21], yrange = [], \
#            marker_style = ['circle', 'circle', 'triangle-down','square', 'diamond', 'cross',  'x-thin', 'cross-thin' ], \
#            width=1000, height=500, path=MYPATH, yanchor='auto', y0_anchor=0.01, xanchor='left', x0_anchor=0.3)


In [144]:
data_b[9][ (data_b[9]['t']-20).abs() < 0.005]

,t,T,alpha
688,19.995846,1.38266,0.988954


In [146]:
tfix = 20
tol = 0.005
list_x = [] #data_w['t']
list_y = [] #data_w['alpha']
for lev in levs:
    try:
        df = data_b[lev]
        list_y.append(abs(df['alpha'][(df['t'] - tfix).abs()< tol].values[0] - data_w['alpha'][data_w['t'] == tfix].values[0]))
        list_x.append(10/2**lev)
    except:
        print("Can\'t read lev=", lev)

slope, intercept, r_value, p_value, std_err = linregress(np.log(list_x), np.log(list_y))
print('Slope:', slope)

plot_graph([list_x], [list_y], names, xtitle=r'$\Large h$', ytitle=r'$\Large\alpha$', image_name='Cylinder_Err_alpha_vs_h.pdf', list_x_fill=[], list_y_fill=[], mode=['markers+lines', 'markers+lines', 'lines', 'lines', 'lines', 'lines'], \
           dash=['solid', 'dot', 'dash', 'longdash', 'longdashdot'], \
           colors=['blue', 'red', 'hsv(120,100,100)', 'green', 'black' ], \
           marker_size=10, xrange =[], yrange = [], \
           marker_style = ['circle', 'circle', 'triangle-down','square', 'diamond', 'cross',  'x-thin', 'cross-thin' ], \
           width=1000, height=500, path=MYPATH, yanchor='auto', y0_anchor=0.01, xanchor='left', x0_anchor=0.7, xlog=True, ylog=True)


Slope: 1.369005225224381
Plot curve number: 0


Write image to file: ../Heat_transfer_Polymerization/Verif/Cylinder_Err_alpha_vs_h.pdf
Successfully generated: ../Heat_transfer_Polymerization/Verif/Cylinder_Err_alpha_vs_h.pdf


In [147]:
tfix = 2.5
list_x = [] #data_w['t']
list_y = [] #data_w['alpha']
for lev in levs:
    df = data_b[lev]
    list_x.append(10/2**lev)
    list_y.append(abs(df['T'][(df['t'] - tfix).abs()< tol].values[0] - data_w['T'][data_w['t'] == tfix].values[0]))

    
slope, intercept, r_value, p_value, std_err = linregress(np.log(list_x), np.log(list_y))
print('Slope:', slope)
plot_graph([list_x], [list_y], names, xtitle=r'$\Large h$', ytitle=r'$\Large T$', image_name='Cylionder_Err_T_vs_h.pdf', list_x_fill=[], list_y_fill=[], mode=['markers+lines', 'markers+lines', 'lines', 'lines', 'lines', 'lines'], \
           dash=['solid', 'dot', 'dash', 'longdash', 'longdashdot'], \
           colors=['blue', 'red', 'hsv(120,100,100)', 'green', 'black' ], \
           marker_size=10, xrange =[], yrange = [], \
           marker_style = ['circle', 'circle', 'triangle-down','square', 'diamond', 'cross',  'x-thin', 'cross-thin' ], \
           width=1000, height=500, path=MYPATH, yanchor='auto', y0_anchor=0.01, xanchor='left', x0_anchor=0.7, xlog=True, ylog=True)


Slope: 0.620689799875235
Plot curve number: 0


Write image to file: ../Heat_transfer_Polymerization/Verif/Cylionder_Err_T_vs_h.pdf
Successfully generated: ../Heat_transfer_Polymerization/Verif/Cylionder_Err_T_vs_h.pdf


# Verification Couette flow

In [175]:
data_w = pd.read_csv("../Heat_transfer_Polymerization/Verif/cylinder_polymerization_wolfram_x=0.5.csv")
data_b = {}
levs = [8,9,10,11]
for lev in levs:
    data_b[lev] = pd.read_csv("../Heat_transfer_Polymerization/Verif/Couette_Jmax=" + str(lev) + "/cylinder_polymerization_basilisk_x=0.csv")
#     data_b[lev] = pd.read_csv("../Heat_transfer_Polymerization/Verif/cylinder_polymerization_basilisk_x=02.csv")


# xz = pd.read_csv('../Heat_transfer_Polymerization/Verif/Couette_Jmax=11_no_mucell_adapt_eps=1e-3/cylinder_polymerization_basilisk_x=0.csv')
# data_b[11] = xz
tmax = df = data_b[9]['t'].max()


In [176]:
list_x = [data_w['t']]
list_y = [data_w['T']]
names = ['Wolfram']
for lev in levs:
    print(lev)
    list_x.append(data_b[lev]['t'])
    list_y.append(data_b[lev]['T'])
    names.append(r'$J_{\max}=' + str(lev) + '$')


plot_graph(list_x, list_y, names, xtitle='t', ytitle='T', image_name='Couette_T_t.pdf', list_x_fill=[], list_y_fill=[], mode=['lines', 'lines', 'lines', 'lines', 'lines', 'lines'], \
           dash=['solid', 'dot', 'dash', 'longdash', 'longdashdot'], \
           colors=['blue', 'red', 'hsv(120,100,100)', 'green', 'black' ], \
           marker_size=2, xrange =[0, tmax], yrange = [], \
           marker_style = ['circle', 'circle', 'triangle-down','square', 'diamond', 'cross',  'x-thin', 'cross-thin' ], \
           width=1000, height=500, path=MYPATH, yanchor='auto', y0_anchor=0.01, xanchor='left', x0_anchor=0.3)

list_x = [data_w['t']]
list_y = [data_w['alpha']]
for lev in levs:
    list_x.append(data_b[lev]['t'])
    list_y.append(data_b[lev]['alpha'])
    
plot_graph(list_x, list_y, names, xtitle=r'$\Large t$', ytitle=r'$\Large\alpha$', image_name='Couette_alpha_t.pdf', list_x_fill=[], list_y_fill=[], mode=['lines', 'lines', 'lines', 'lines', 'lines', 'lines'], \
           dash=['solid', 'dot', 'dash', 'longdash', 'longdashdot'], \
           colors=['blue', 'red', 'hsv(120,100,100)', 'green', 'black' ], \
           marker_size=2, xrange =[0, tmax], yrange = [], \
           marker_style = ['circle', 'circle', 'triangle-down','square', 'diamond', 'cross',  'x-thin', 'cross-thin' ], \
           width=1000, height=500, path=MYPATH, yanchor='auto', y0_anchor=0.01, xanchor='left', x0_anchor=0.3)


8
9
10
11
Plot curve number: 0
Plot curve number: 1
Plot curve number: 2
Plot curve number: 3
Plot curve number: 4


Write image to file: ../Heat_transfer_Polymerization/Verif/Couette_T_t.pdf
Successfully generated: ../Heat_transfer_Polymerization/Verif/Couette_T_t.pdf
Plot curve number: 0
Plot curve number: 1
Plot curve number: 2
Plot curve number: 3
Plot curve number: 4


Write image to file: ../Heat_transfer_Polymerization/Verif/Couette_alpha_t.pdf
Successfully generated: ../Heat_transfer_Polymerization/Verif/Couette_alpha_t.pdf


In [172]:
tfix = 0.6
list_x = [] #data_w['t']
list_y = [] #data_w['alpha']
for lev in levs:
    try:
        df = data_b[lev]
        list_y.append(abs(df['alpha'][(df['t'] - tfix).abs()< tol].values[0] - data_w['alpha'][data_w['t'] == tfix].values[0]))
        list_x.append(10/2**lev)
    except:
        print('Can\'t read lev=', lev)
    
slope, intercept, r_value, p_value, std_err = linregress(np.log(list_x), np.log(list_y))
print('Slope:', slope)
plot_graph([list_x], [list_y], names, xtitle=r'$\Large h$', ytitle=r'$\Large E_\alpha$', image_name='Couette_Err_alpha_vs_h.pdf', list_x_fill=[], list_y_fill=[], mode=['markers+lines', 'markers+lines', 'lines', 'lines', 'lines', 'lines'], \
           dash=['solid', 'dot', 'dash', 'longdash', 'longdashdot'], \
           colors=['blue', 'red', 'hsv(120,100,100)', 'green', 'black' ], \
           marker_size=10, xrange =[], yrange = [], \
           marker_style = ['circle', 'circle', 'triangle-down','square', 'diamond', 'cross',  'x-thin', 'cross-thin' ], \
           width=1000, height=500, path=MYPATH, yanchor='auto', y0_anchor=0.01, xanchor='left', x0_anchor=0.7, xlog=True, ylog=True)


Slope: 0.8991526913763815
Plot curve number: 0


Write image to file: ../Heat_transfer_Polymerization/Verif/Couette_Err_alpha_vs_h.pdf
Successfully generated: ../Heat_transfer_Polymerization/Verif/Couette_Err_alpha_vs_h.pdf


In [174]:
tfix = 2.5
list_x = [] #data_w['t']
list_y = [] #data_w['alpha']
for lev in levs:
    try:
        df = data_b[lev]
        list_y.append(abs(df['T'][(df['t'] - tfix).abs()< tol].values[0] - data_w['T'][data_w['t'] == tfix].values[0]))
        list_x.append(10/2**lev)
    except:
        print('Can\'t read lev=', lev)

    
slope, intercept, r_value, p_value, std_err = linregress(np.log(list_x), np.log(list_y))
print('Slope:', slope)
plot_graph([list_x], [list_y], names, xtitle=r'$\Large h$', ytitle=r'$\Large E_T$', image_name='Couette_Err_T_vs_h.pdf', list_x_fill=[], list_y_fill=[], mode=['markers+lines', 'markers+lines', 'lines', 'lines', 'lines', 'lines'], \
           dash=['solid', 'dot', 'dash', 'longdash', 'longdashdot'], \
           colors=['blue', 'red', 'hsv(120,100,100)', 'green', 'black' ], \
           marker_size=10, xrange =[], yrange = [], \
           marker_style = ['circle', 'circle', 'triangle-down','square', 'diamond', 'cross',  'x-thin', 'cross-thin' ], \
           width=1000, height=500, path=MYPATH, yanchor='auto', y0_anchor=0.01, xanchor='left', x0_anchor=0.7, xlog=True, ylog=True)


Can't read lev= 10
Slope: 0.49217998147599185
Plot curve number: 0


Write image to file: ../Heat_transfer_Polymerization/Verif/Couette_Err_T_vs_h.pdf
Successfully generated: ../Heat_transfer_Polymerization/Verif/Couette_Err_T_vs_h.pdf


In [79]:
data_b[10]['t'].max()

2.82

In [114]:
list_y

[0.0012953862487117274, 0.000625386248711779]